In [3]:
import os
import io
import sys
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
from IPython.display import Markdown, display, update_display
import gradio as gr
import subprocess

In [28]:
import os
os.environ['PATH'] += ':/home/chandan/anaconda3/bin'
os.environ['SHELL'] = '/bin/bash'


In [5]:
load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

In [6]:
openai=OpenAI()
model='gpt-4o'

In [7]:
system_message = "You are an assistant that reimplements Python code in high performance C++ for an Ryzen5 processor. "
system_message += "Respond only with C++ code; use comments sparingly and do not provide any explanation other than occasional comments. "
system_message += "The C++ response needs to produce an identical output in the fastest possible time."

In [59]:
def user_prompt(python):
    prompt = "Rewrite this Python code in C++ with the fastest possible implementation that produces identical output in the least time. "
    prompt += "Respond only with C++ code; do not explain your work other than a few comments. "
    prompt += "Pay attention to number types to ensure no int overflows. Remember to #include all necessary C++ packages such as iomanip.\n\n"
    prompt += python
    prompt += "write it in proper executable cpp format don't write anything else not even comments and your code should be directly executable"
    return prompt

In [60]:
def message_for(python):
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt(python)}
    ]

In [73]:
def write_output(cpp):
    code = cpp.replace("```cpp","").replace("```","")
    with open("optimized.cpp", "w") as f:
        f.write(code)

In [68]:
def optimize_gpt(python):
    stream=openai.chat.completions.create(model=model, messages=message_for(python), stream=True)
    reply=""
    for chunk in stream:
        frag=chunk.choices[0].delta.content or ""
        reply+=frag
        print(frag, end='', flush=True)
    write_output(reply)

In [69]:
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [70]:
exec(pi)

Result: 3.141592658589
Execution Time: 9.320251 seconds


In [71]:
optimize_gpt(pi)

```cpp
#include <iostream>
#include <iomanip>
#include <chrono>

double calculate(int iterations, int param1, int param2) {
    double result = 1.0;
    for (int i = 1; i <= iterations; ++i) {
        int j = i * param1 - param2;
        result -= (1.0 / j);
        j = i * param1 + param2;
        result += (1.0 / j);
    }
    return result;
}

int main() {
    auto start_time = std::chrono::high_resolution_clock::now();
    double result = calculate(100000000, 4, 1) * 4;
    auto end_time = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double> execution_time = end_time - start_time;

    std::cout << std::fixed << std::setprecision(12);
    std::cout << "Result: " << result << std::endl;
    std::cout << "Execution Time: " << execution_time.count() << " seconds" << std::endl;

    return 0;
}
```

In [74]:
!g++ -O3 -std=c++17 -march=x86-64 -o optimized optimized.cpp && ./optimized



Result: 3.141592658589
Execution Time: 0.217682148000 seconds
